## 埋込モデル

### 必要なライブラリのインストール

In [ ]:
%pip install -r ../requirements.txt

### 環境変数の設定

In [1]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

### 埋め込みモデル


In [3]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    model="embed-multilingual-v3.0"
)

### Document Loader

`../docs/*.txt` に格納されているテキストデータを読み込み、LangChain の Document へ変換します。

In [4]:
import glob
from langchain.document_loaders import TextLoader

files = glob.glob("../docs/*.txt")
documents = []

for file in files:
    loader = TextLoader(file_path=file)
    document = loader.load()
    documents.extend(document)

### Vector Store

In [5]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

vector_store.add_documents(documents=documents)

['9bd3f14e-b51e-4077-a8d2-87bc3cdc6053',
 '6de8a8aa-5927-4bac-aebc-ba5fdd12c6f8']

自然言語を用いた類似度検索を行います。

In [8]:
result = vector_store.similarity_search(query="カルビクッパ")
result

[Document(metadata={'source': '../docs/karubikuppa.txt'}, page_content='BB流カルビクッパの作り方です。\n\nまずは、以下の材料を用意します。なお、材料は2人分です。\n\n- 牛肉: 50g\n- 大根: 半分\n- 人参: 1本\n- シイタケ: 2-3個\n- ニンニク: 半分\n- もやし: 1袋\n- ニラ: 半分\n- ぜんまい: 40g\n    - BB流ポイント 「入れなくても良いですが、入れると@<ttb>{めちゃ旨}です！」\n- つゆ (*)\n    - 水: 800cc\n    - コチュジャン: 大さじ2\n        - BB流ポイント「入れれば入れるだけ美味しい！」\n    - ウェイパー: 大さじ2\n    - ごま油: 大さじ2\n        - BB流ポイント「入れれば入れるだけ美味しい！」\n    - 酒: 大さじ2\n    - 醤油: 大さじ2\n    - 砂糖: 大さじ2\n- 塩コショウ:適量\n- 卵: 2個\n\nごま油でニンニクを炒め、香りが出たら牛肉を炒めます。つゆ（*）をいれ、ニラともやし以外を入れてひと煮立ちさせます。\n最後にニラともやしを入れてある程度シャキシャキ感を残しつつ、卵をふんわりさせたらBB流カルビクッパの完成です！\nなお、卵をふんわりさせるコツはつゆを限界まで沸騰させることです！\n'),
 Document(metadata={'source': '../docs/kinoko-marine.txt'}, page_content='BB流きのこマリネの作り方です。\n\nまずは、以下の材料を用意します。なお、材料は2人分です。\n\n- お好きなきのこ\n    - 複数種類用意するととても美味しいです\n    - BB的には、エリンギ、椎茸、マッシュルーム、しめじ、舞茸、平茸などがおすすめです\n- バター（無塩）: 10g\n- にんにく: 1個\n- マリネ用のたれ (*)\n    - マスタード: 大さじ2\n    - 白ワインビネガー: 大さじ1~2\n    - ケッパー: 15粒程度\n- パセリ: お好きな方はたくさん\n\nフライパンにオリーブオイルをひき、強火できのこ

### Retriever

In [9]:
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f95005a2de0>)

### シンプルなRAGを用いたアプリケーション

In [10]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

from langchain_cohere.chat_models import ChatCohere

chat = ChatCohere(
    model="command-r-plus"
)
retriever = vector_store.as_retriever()

prompt_template = PromptTemplate.from_template(template="""
以下のコンテキストに基づいて質問に対する回答を作成してください。

## コンテキスト

{context}

## 質問

{question}
""")

chain = (
    {"question": RunnablePassthrough(), "context": retriever}
    | prompt_template
    | chat
    | StrOutputParser()
)

response = chain.stream("カルビクッパってどうやって作るのでしょうか？")

for chunk in response:
    print(chunk, end="")

BB流カルビクッパの作り方は、まずごま油でニンニクを炒め、香りが出たら牛肉を炒めます。つゆ（水、コチュジャン、ウェイパー、ごま油、酒、醤油、砂糖を混ぜたもの）を入れ、細かく切った野菜（大根、人参、シイタケ、ニラ、もやし）とぜんまいを加えます。最後にニラともやしを入れてシャキシャキ感を残しつつ、卵をふんわりとさせたら完成です。卵をふんわりさせるコツは、つゆを限界まで沸騰させることです。